### Importing modules

In [1]:
import hoomd
import hoomd.hpmc as hpmc
import coxeter
import ipywidgets as widgets
import gsd, gsd.hoomd
import plato.draw.vispy as draw
import numpy as np

### Choosing parameters for our simulation

This includes choosing the two shapes we will be using, the ratio between the shapes, and the total number of shapes in the simulation box.

In [2]:
getShape = coxeter.families.RegularNGonFamily()

def make3D(vertex_list):
    to_add = np.transpose([np.zeros(len(vertex_list))])
    vertex_list = np.concatenate([vertex_list,to_add],axis=1)
    return vertex_list

verticesA = make3D(getShape.make_vertices(4))
verticesB = make3D(getShape.make_vertices(5))

In [16]:
verticesB

array([[ 0.64852517,  0.        ,  0.        ],
       [ 0.2004053 ,  0.61678408,  0.        ],
       [-0.52466788,  0.38119353,  0.        ],
       [-0.52466788, -0.38119353,  0.        ],
       [ 0.2004053 , -0.61678408,  0.        ]])

In [3]:
# Choose a fraction between 0 and 1
# which will represent the percent of shapes
# that are Shape 1
# shape_ratio = 0 means all the shapes will be Shape 2

shape_ratio = widgets.FloatSlider(
    min=0.00, 
    max=1.00, 
    step=0.01, 
    description='Shape ratio:', 
    value=0.50,
    readout_format='.2f')

display(shape_ratio)

FloatSlider(value=0.5, description='Shape ratio:', max=1.0, step=0.01)

In [4]:
# Setting number of particles along one side of our simulation box
# which is the square root of the total number of particles
num_particles_sqrt = 20

In [ ]:
#setting colors

### Initializing the simulation

In [ ]:
#making & visualizing the unit cell

In [5]:
hoomd.context.initialize("--mode=cpu");

system = hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=2,
                                                                  a1=[3,0,0],
                                                                  a2=[0,6,0],
                                                                  a3=[0,0,1],
                                                                  dimensions=2,
                                                                  position=[[0,0,0],
                                                                            [0,3,0]],
                                                                  type_name=["A","B"]),
                                n=[num_particles_sqrt,int(1/2*num_particles_sqrt)])

snap = system.take_snapshot()

snap.particles.types = ["A","B"]

for p in range(int(shape_ratio.value*(num_particles_sqrt**2))):
    snap.particles.typeid[p]=0
for p in range(int(shape_ratio.value*(num_particles_sqrt**2)),num_particles_sqrt**2):
    snap.particles.typeid[p]=1
    
system.restore_snapshot(snap)

HOOMD-blue 2.9.7 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 08/09/2021
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
-----
You are using HPMC. Please cite the following:
* J A Anderson, M E Irrgang, and S C Glotzer. "Scalable Metropolis Monte Carlo
  for simulation of hard shapes", Computer Physics Communications 204 (2016) 21
  --30
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 400 particles


In [15]:
#visualizing our initial frame with plato
primA = draw.Polygons(vertices = verticesA[:,:2])
primB = draw.Polygons(vertices = verticesB[:,:2])
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2)
scene = draw.Scene([primA, box_prim], zoom=.5, clip_scale=5)
scene.add_primitive(primB)
scene.show()

box = [snap.box.Lx, snap.box.Ly, snap.box.Lz, snap.box.xy, snap.box.xz, snap.box.yz]
for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
    setattr(box_prim, name, val)  

primA.positions = snap.particles.position[snap.particles.typeid == 0,:2]
primB.positions = snap.particles.position[snap.particles.typeid == 1,:2]

primA.orientations = snap.particles.orientation
primB.orientations = snap.particles.orientation

primA.diameters = np.full(len(snap.particles.position), 1)
primB.diameters = np.full(len(snap.particles.position), 1)

colorsA = np.ones((len(primA.positions), 4))
colorsA[:, :3] = np.float32(np.divide([255, 50, 150], 255)) #pink
primA.colors = colorsA

colorsB = np.ones((len(primB.positions), 4))
colorsB[:, :3] = np.float32(np.divide([0, 200, 100], 255)) #green
primB.colors = colorsB

scene.render()

VispyWidget(height=600, width=800)

### Running the simulation

In [7]:
mc = hpmc.integrate.convex_polygon(seed=np.random.randint(1,1e6), d=0.5, a=0.5, move_ratio=0.5)
mc.shape_param.set('A', vertices=verticesA)
mc.shape_param.set('B', vertices=verticesB)

boxMC = hpmc.update.boxmc(mc, hoomd.variant.linear_interp(points=[[0,1],[1e5,20]],zero=0), 
                          np.random.randint(1,1e6))

boxMC.volume(delta=1.0, weight=1.0)

particle_tuner = hpmc.util.tune(obj=mc, tunables=['d','a'], target=0.2)
box_tuner = hpmc.util.tune_npt(obj=boxMC, tunables=['dV'],  target=0.2)


all=hoomd.group.all()

In [8]:
traj = hoomd.dump.gsd(filename='traj3.gsd',period=int(100),group=all,overwrite=True)
traj.dump_shape(mc)

In [9]:
hoomd.run(1e6)

** starting run **
Time 00:00:45 | Step 11839 / 1000000 | TPS 1183.83 | ETA 00:13:54
Time 00:00:55 | Step 21319 / 1000000 | TPS 947.931 | ETA 00:17:12
Time 00:01:05 | Step 29957 / 1000000 | TPS 863.739 | ETA 00:18:43
Time 00:01:15 | Step 37645 / 1000000 | TPS 768.71 | ETA 00:20:51
Time 00:01:25 | Step 45835 / 1000000 | TPS 818.985 | ETA 00:19:25
Time 00:01:35 | Step 53878 / 1000000 | TPS 804.298 | ETA 00:19:36
Time 00:01:45 | Step 62434 / 1000000 | TPS 855.52 | ETA 00:18:15
Time 00:01:55 | Step 71890 / 1000000 | TPS 945.571 | ETA 00:16:21
Time 00:02:05 | Step 81159 / 1000000 | TPS 926.87 | ETA 00:16:31
Time 00:02:15 | Step 90799 / 1000000 | TPS 963.959 | ETA 00:15:43
Time 00:02:25 | Step 100662 / 1000000 | TPS 986.215 | ETA 00:15:11
Time 00:02:35 | Step 110713 / 1000000 | TPS 1005.06 | ETA 00:14:44
Time 00:02:45 | Step 120796 / 1000000 | TPS 1008.23 | ETA 00:14:32
Time 00:02:55 | Step 130540 / 1000000 | TPS 974.391 | ETA 00:14:52
Time 00:03:05 | Step 140436 / 1000000 | TPS 989.569 | ET

In [ ]:
#graph volume over time -- Rachael
#graph packing fraction over time -- Rachael

### Visualizing your simulation

In [10]:
def getFrameCount(fname):
    #inputs: fname, the filename (ex: 'dump.gsd')
    #outputs: len(traj), number of frames in simulation
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

In [17]:
primA = draw.Polygons(vertices = verticesA[:,:2])
primB = draw.Polygons(vertices = verticesB[:,:2])
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2)
scene = draw.Scene([primA, box_prim], zoom=.5, clip_scale=5)
scene.add_primitive(primB)
scene.show()

filename = 'traj3.gsd'
frame_num = getFrameCount(filename)

@widgets.interact(frame_index=(0, frame_num-1, 1))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)  
            
        primA.positions = frame.particles.position[frame.particles.typeid == 0,:2]
        primB.positions = frame.particles.position[frame.particles.typeid == 1,:2]
        
        primA.orientations = frame.particles.orientation
        primB.orientations = frame.particles.orientation
        
        primA.diameters = np.full(len(frame.particles.position), 1)
        primB.diameters = np.full(len(frame.particles.position), 1)
        
        colorsA = np.ones((len(primA.positions), 4))
        colorsA[:, :3] = np.float32(np.divide([255, 50, 150], 255)) #pink
        primA.colors = colorsA
        
        colorsB = np.ones((len(primB.positions), 4))
        colorsB[:, :3] = np.float32(np.divide([0, 200, 100], 255)) #green
        primB.colors = colorsB
    scene.render()

VispyWidget(height=600, width=800)

interactive(children=(IntSlider(value=0, description='frame_index', max=9999), Output()), _dom_classes=('widge…